## Descriptions Courtes des Fonctions du Projet

### 1. `trouver_routes_proches`
Cette fonction identifie les routes les plus proches d'un point géographique spécifique. Elle utilise une API WFS pour récupérer les données géospatiales et calculer la proximité des routes par rapport au point donné.

### 2. `calculer_itineraire_et_distance`
Cette fonction calcule l'itinéraire et la distance entre deux points géographiques pour un piéton, en utilisant le service de calcul d'itinéraire de l'IGN. Elle gère les configurations des requêtes et extrait les données utiles, comme la distance et la géométrie du trajet.

### 3. `obtenir_combinaisons_itineraires`
Cette fonction détermine toutes les combinaisons possibles d'itinéraires entre un point de départ et un point d'arrivée en explorant plusieurs routes proches. Elle calcule et enregistre les itinéraires pour chaque combinaison de routes, utilisant les fonctions précédentes pour optimiser la planification du trajet.

Ces fonctions servent de base pour des applications de navigation et d'analyse géospatiale, permettant une exploration détaillée des itinéraires et des distances entre différents points géographiques.




In [ ]:
import requests
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from io import BytesIO
import pandas as pd
import heapq

In [ ]:
pd.options.display.float_format = '{:.6f}'.format  # Configure le format des nombres flottants pour éviter la notation scientifique

### Description de la fonction `trouver_routes_proches`

La fonction `trouver_routes_proches` est utilisée pour identifier les routes les plus proches d'un point géographique donné. Elle utilise une API WFS pour récupérer les données géospatiales et calculer la proximité des routes.

#### Paramètres

- `point` : tuple(float, float)
  - Les coordonnées (latitude, longitude) du point d'intérêt.
- `bbox_size` : float
  - La taille de la boîte englobante (bbox) autour du point pour la recherche des routes.
- `url_base` : str
  - L'URL de base de l'API WFS utilisée pour la récupération des données.
- `couche` : str
  - Le nom de la couche géospatiale à interroger via l'API WFS.
- `nombre_routes` : int (optionnel, par défaut à 5)
  - Le nombre de routes les plus proches à retourner.

#### Fonctionnement

1. **Définition de la boîte englobante (bbox)** :
   - Création d'une bbox autour du point avec les dimensions spécifiées par `bbox_size`.
   - Conversion des coordonnées du bbox en chaîne de caractères au format requis par l'API.

2. **Paramétrage et requête API** :
   - Définition des paramètres requis par l'API WFS, y compris le type de requête, la version du service, la boîte englobante, etc.
   - Envoi de la requête à l'URL spécifiée et récupération des données au format JSON.

3. **Traitement des données** :
   - Conversion des données JSON en GeoDataFrame.
   - Calcul de la distance entre le point donné et chaque route trouvée en utilisant une projection précise (EPSG:2154).
   - Récupération et conversion des points les plus proches en WGS84 pour chaque route.

4. **Sélection des routes** :
   - Utilisation d'un tas pour trier et récupérer les routes les plus proches en fonction de la distance calculée.
   - Sélection des `nombre_routes` premières routes les plus proches.

5. **Construction du résultat** :
   - Création d'un DataFrame contenant les détails des routes sélectionnées, y compris la distance et les coordonnées en WGS84.

#### Retour

- `pd.DataFrame`
  - Un DataFrame contenant les routes les plus proches avec leurs détails, ou vide en cas d'erreur.

#### Gestion des erreurs

- Si l'API retourne un code de statut différent de 200, un message d'erreur est affiché et un DataFrame vide est retourné.


In [ ]:
def trouver_routes_proches(point, bbox_size, url_base, couche, nombre_routes=5):
    lat, lon = point
    bbox = f"{lon - bbox_size},{lat - bbox_size},{lon + bbox_size},{lat + bbox_size},EPSG:4326"

    params = {
        'service': 'WFS',
        'version': '2.0.0',
        'request': 'GetFeature',
        'typeName': couche,
        'srsName': 'EPSG:4326',
        'outputFormat': 'application/json',
        'bbox': bbox
    }

    response = requests.get(url_base, params=params)
    if response.status_code == 200:
        routes = gpd.read_file(BytesIO(response.content))
        routes = routes.to_crs(epsg=2154)  # Conversion à EPSG:2154 pour calcul précis
        point_geom = Point(lon, lat)
        point_geom = gpd.GeoSeries([point_geom], crs='epsg:4326').to_crs(epsg=2154)[0]

        distances_et_details = []
        for route in routes.itertuples():
            nearest = nearest_points(point_geom, route.geometry)[1]
            distance = point_geom.distance(nearest)

            # Convertir les points les plus proches en WGS84
            nearest_wgs84 = gpd.GeoSeries([nearest], crs='epsg:2154').to_crs(epsg=4326)[0]

            details = [
                distance,
                nearest_wgs84.x,  # Longitude en WGS84
                nearest_wgs84.y,  # Latitude en WGS84
            ] + list(route[1:-1])  # Ajouter les autres propriétés
            heapq.heappush(distances_et_details, details)

        # Sélection des n premières routes les plus proches
        resultats = heapq.nsmallest(nombre_routes, distances_et_details)

        # Créer un DataFrame à partir des résultats
        columns = ['Distance', 'Longitude', 'Latitude'] + [col for col in routes.columns if col != 'geometry']
        return pd.DataFrame(resultats, columns=columns)
    else:
        print(f"Erreur lors de la récupération des données : {response.status_code}")
        print(response.text)
        return pd.DataFrame()


In [ ]:
# Usage de la fonction
url_base = 'https://wxs.ign.fr/geoportail/wfs'
nom_couche_correct = 'BDTOPO_V3:troncon_de_route'
point_interet = (43.876744 , 2.1078)  # Coordonnées du pont
bbox_size = 0.05

In [ ]:
df_routes = trouver_routes_proches(point_interet, bbox_size, url_base, nom_couche_correct)
print(df_routes)

    Distance  Longitude  Latitude                         id  \
0  62.827002   2.108130 43.876232   troncon_de_route.4856495   
1 211.679951   2.108411 43.874891  troncon_de_route.16206923   
2 291.733971   2.111392 43.877119  troncon_de_route.18147534   
3 317.704713   2.105716 43.879174  troncon_de_route.18147578   
4 331.869118   2.103790 43.877455  troncon_de_route.18147579   

                     cleabs              nature nom_collaboratif_gauche  \
0  TRONROUT0000000052101290  Route à 1 chaussée               CRESSENTY   
1  TRONROUT0000000052101317              Chemin                    None   
2  TRONROUT0000000052100121  Route à 1 chaussée          CHE DU VERDIER   
3  TRONROUT0000000052100140  Route à 1 chaussée          RTE DE RIEUMAS   
4  TRONROUT0000000052100141  Route à 1 chaussée          RTE DE RIEUMAS   

  nom_collaboratif_droite importance  fictif  ... ouvrage_d_art_limitant_dfci  \
0               CRESSENTY          5   False  ...                        None   
1 

**Affichage des resultats**

In [ ]:
import folium

In [ ]:
def afficher_sur_carte(df, point_depart):
    # Créer une carte centrée autour du point de départ
    m = folium.Map(location=[point_depart[0], point_depart[1]], zoom_start=14)

    # Ajouter un marqueur pour le point de départ
    folium.Marker(
        [point_depart[0], point_depart[1]],
        popup='Point de départ',
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

    # Ajouter un marqueur pour chaque point de route trouvé et tracer une ligne depuis le point de départ
    for idx, row in df.iterrows():
        folium.Marker(
            [row['Latitude'], row['Longitude']],
            popup=f'Distance: {row["Distance"]:.2f} m',
            icon=folium.Icon(color='blue', icon='road')
        ).add_to(m)

        # Tracer une ligne du point de départ au point actuel
        folium.PolyLine(
            locations=[(point_depart[0], point_depart[1]), (row['Latitude'], row['Longitude'])],
            color='blue',
            weight=2,
            opacity=0.5
        ).add_to(m)

    # Afficher la carte
    return m

In [ ]:
# Exemple d'utilisation
point_interet = (43.876744 , 2.1078)  # Coordonnées du point
df_routes = trouver_routes_proches(point_interet, 0.05, 'https://wxs.ign.fr/geoportail/wfs', 'BDTOPO_V3:troncon_de_route')
m = afficher_sur_carte(df_routes, point_interet)
m

### Description de la fonction `calculer_itineraire_et_distance`

La fonction `calculer_itineraire_et_distance` permet de calculer l'itinéraire et la distance entre deux points géographiques pour un piéton, en utilisant le service de calcul d'itinéraire de l'IGN.

#### Paramètres

- `point_depart` : tuple(float, float)
  - Les coordonnées (latitude, longitude) du point de départ.
- `point_arrivee` : tuple(float, float)
  - Les coordonnées (latitude, longitude) du point d'arrivée.

#### Fonctionnement

1. **Configuration de l'URL et des paramètres de la requête** :
   - Définition de l'URL du service IGN pour le calcul d'itinéraire.
   - Configuration des paramètres de la requête, incluant le profil de déplacement (`pedestrian`), le type d'optimisation (`shortest`), et les formats des données retournées (`geojson` pour la géométrie).

2. **Envoi de la requête et gestion des réponses** :
   - Envoi de la requête HTTP GET avec les paramètres définis.
   - Gestion des réponses HTTP potentiellement erronées avec `response.raise_for_status()`.
   - Extraction des données utiles de la réponse, telles que la distance totale parcourue et la géométrie de l'itinéraire.

3. **Extraction et retour des données** :
   - La distance est extraite et retournée, permettant une évaluation directe du parcours.
   - La géométrie de l'itinéraire est également retournée pour une utilisation éventuelle dans des visualisations ou des analyses supplémentaires.

#### Retour

- `tuple`
  - Un tuple contenant:
    - `data` : dict, le JSON complet de la réponse de l'API.
    - `distance` : str, la distance calculée ou un message indiquant que la distance n'est pas disponible.
    - `geometry` : dict, la géométrie de l'itinéraire en format geojson.

#### Gestion des erreurs

- En cas d'erreur lors de la requête (e.g., problème de réseau, réponse erronée de l'API), la fonction affiche un message d'erreur et retourne un tuple avec `None` pour la donnée, un message d'erreur pour la distance, et un dictionnaire vide pour la géométrie.



In [ ]:
def calculer_itineraire_et_distance(point_depart, point_arrivee):
    url = "https://data.geopf.fr/navigation/itineraire?"

    params = {
        'resource': 'bdtopo-osrm',
        'profile': 'pedestrian',
        'optimization': 'shortest',
        'start': f"{point_depart[1]},{point_depart[0]}",
        'end': f"{point_arrivee[1]},{point_arrivee[0]}",
        'geometryFormat': 'geojson'
    }

    try:
        response = requests.get(url, params=params, timeout=10)  # Ajout d'un timeout
        response.raise_for_status()  # Gérer les erreurs HTTP
        data = response.json()
        distance = data.get('distance', 'Non disponible')
        geometry = data.get('geometry', {})
        return data, distance, geometry
    except requests.RequestException as e:
        print(f"Erreur lors de la requête: {e}")
        return None, 'Erreur lors de la récupération de la distance', {}


In [ ]:
# Exemple d'utilisation
point_depart = (43.852683 , 2.06575)  # Coordonnées du point
point_arrivee = (43.793016 , 1.972391)  # Coordonnées du point

itineraire, distance, geometrie = calculer_itineraire_et_distance(point_depart, point_arrivee)


In [ ]:
print (distance)

12201.8


**Affichage des resultats**

In [ ]:

# Créer une carte centrée sur le point de départ
m = folium.Map(location=point_arrivee, zoom_start=15)

# Ajouter l'itinéraire à la carte
if itineraire and 'geometry' in data:
    folium.GeoJson(itineraire['geometry'], name='geojson').add_to(m)

# Afficher la carte
m

### Description de la fonction `obtenir_combinaisons_itineraires`

La fonction `obtenir_combinaisons_itineraires` est conçue pour identifier et calculer les itinéraires possibles entre un point de départ et un point d'arrivée en utilisant plusieurs routes potentielles à proximité de ces points.

#### Paramètres

- `point_depart` : tuple(float, float)
  - Coordonnées (latitude, longitude) du point de départ.
- `point_arrivee` : tuple(float, float)
  - Coordonnées (latitude, longitude) du point d'arrivée.
- `bbox_size` : float
  - Taille de la boîte englobante (bbox) utilisée pour délimiter la recherche des routes proches.
- `url_base` : str
  - URL de base de l'API pour les requêtes des routes proches.
- `couche` : str
  - Nom de la couche cartographique sur laquelle effectuer les requêtes.
- `nombre_routes` : int (optionnel, par défaut à 5)
  - Nombre de routes proches à considérer pour chaque point.

#### Fonctionnement

1. **Identification des routes proches** :
   - Pour le point de départ et d'arrivée, la fonction utilise `trouver_routes_proches` pour obtenir les routes les plus proches selon le nombre spécifié par `nombre_routes`.

2. **Calcul des itinéraires** :
   - Pour chaque combinaison possible des routes proches trouvées pour le point de départ et d'arrivée, la fonction calcule l'itinéraire à l'aide de `calculer_itineraire_et_distance`.

3. **Enregistrement des combinaisons** :
   - Chaque itinéraire calculé entre une route de départ et une route d'arrivée est enregistré avec les distances associées et la géométrie de l'itinéraire.

#### Retour

- `pd.DataFrame`
  - Un DataFrame contenant les informations sur les itinéraires calculés, y compris la distance au départ, la distance à l'arrivée, la distance de l'itinéraire calculée, et la géométrie du trajet.




In [ ]:
def obtenir_combinaisons_itineraires(point_depart, point_arrivee, bbox_size, url_base, couche, nombre_routes=5):
    # Trouver les routes proches pour le point de départ
    routes_depart = trouver_routes_proches(point_depart, bbox_size, url_base, couche, nombre_routes)
    # Trouver les routes proches pour le point d'arrivée
    routes_arrivee = trouver_routes_proches(point_arrivee, bbox_size, url_base, couche, nombre_routes)

    # Liste pour stocker les résultats finaux
    combinaisons = []

    # Calculer l'itinéraire pour chaque combinaison des routes de départ et d'arrivée
    for _, route_depart in routes_depart.iterrows():
        point_route_depart = (route_depart['Latitude'], route_depart['Longitude'])

        for _, route_arrivee in routes_arrivee.iterrows():
            point_route_arrivee = (route_arrivee['Latitude'], route_arrivee['Longitude'])

            # Calculer l'itinéraire entre les deux points de route
            itineraire, distance_itineraire, geometry = calculer_itineraire_et_distance(point_route_depart, point_route_arrivee)

            # Enregistrer les détails dans la liste
            combinaisons.append({
                'RouteDepart': route_depart['Distance'],  # Utilisation de la distance pour 'RouteDepart'
                'DistanceItineraire': distance_itineraire,
                'RouteArrivee': route_arrivee['Distance'],  # Utilisation de la distance pour 'RouteArrivee'
                'Trajet': geometry  # Ajouter la géométrie du trajet
            })

    # Convertir la liste des résultats en DataFrame pour une manipulation plus facile
    return pd.DataFrame(combinaisons)


In [ ]:
# Exemple d'utilisation de la fonction
point_depart = (43.852683 , 2.06575)  # Coordonnées
point_arrivee = (43.793016 , 1.972391)  # Coordonnées
bbox_size = 0.05
url_base = 'https://wxs.ign.fr/geoportail/wfs'
couche = 'BDTOPO_V3:troncon_de_route'

resultats = obtenir_combinaisons_itineraires(point_depart, point_arrivee, bbox_size, url_base, couche)
print(resultats)

    RouteDepart  DistanceItineraire  RouteArrivee  \
0     91.654526        12202.500000    145.045633   
1     91.654526        12145.200000    147.663544   
2     91.654526        12095.700000    183.681832   
3     91.654526        12241.100000    280.575794   
4     91.654526        12698.100000    288.501464   
5    170.956768        12041.200000    145.045633   
6    170.956768        11983.900000    147.663544   
7    170.956768        11934.400000    183.681832   
8    170.956768        12079.800000    280.575794   
9    170.956768        12536.800000    288.501464   
10   178.044517        12272.900000    145.045633   
11   178.044517        12215.600000    147.663544   
12   178.044517        12166.100000    183.681832   
13   178.044517        12311.500000    280.575794   
14   178.044517        12768.500000    288.501464   
15   186.739094        11987.900000    145.045633   
16   186.739094        11930.600000    147.663544   
17   186.739094        11881.100000    183.681

**Affichage des resultats**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def afficher_itineraires_sur_carte(df, point_depart, point_arrivee):
    # Créer une carte centrée autour du point de départ
    m = folium.Map(location=[point_depart[0], point_depart[1]], zoom_start=13)

    # Générer des couleurs distinctes pour chaque itinéraire
    colors = plt.colormaps['tab20'](range(df.shape[0]))
    hex_colors = [mcolors.to_hex(color) for color in colors]  # Convertir les couleurs en format hexadécimal

    # Ajouter chaque itinéraire à la carte
    for idx, row in df.iterrows():
        if pd.notna(row['Trajet']):
            trajet = folium.GeoJson(
                data=row['Trajet'],  # Utiliser directement le dictionnaire GeoJSON
                style_function=lambda _, color=hex_colors[idx]: {
                    'color': color,
                    'weight': 3,
                    'opacity': 0.7
                }
            ).add_to(m)

            # Supposer que les coordonnées sont déjà sous forme de listes de listes pour les lignes
            coordinates = row['Trajet']['coordinates']
            start_point = coordinates[0]
            end_point = coordinates[-1]

            # Ajouter des marqueurs pour les points de départ et d'arrivée de l'itinéraire
            folium.Marker(
                location=start_point[::-1],  # Inverser les coordonnées si nécessaire
                popup=f'Début de Trajet (Index: {idx})',
                icon=folium.Icon(color='blue', icon='play')
            ).add_to(m)

            folium.Marker(
                location=end_point[::-1],  # Inverser les coordonnées si nécessaire
                popup=f'Fin de Trajet (Index: {idx})',
                icon=folium.Icon(color='darkred', icon='stop')
            ).add_to(m)

    # Marquer le point de départ
    folium.Marker(
        location=point_depart,
        popup='Point de Départ',
        icon=folium.Icon(color='green', icon='star')
    ).add_to(m)

    # Marquer le point d'arrivée
    folium.Marker(
        location=point_arrivee,
        popup='Point d\'Arrivée',
        icon=folium.Icon(color='red', icon='flag')
    ).add_to(m)

    # Afficher la carte
    return m


In [ ]:
afficher_itineraires_sur_carte(resultats, point_depart, point_arrivee)